In [1]:
from abc import ABC, abstractmethod
import json
from logging import DEBUG, StreamHandler, getLogger
import pprint
import random
import re

from openai import OpenAI

# from vllm import LLM, SamplingParams

In [2]:
# Set logging level to DEBUG.
logger = getLogger(__name__)
logger.setLevel(DEBUG)
handler = StreamHandler()
handler.setLevel(DEBUG)
logger.addHandler(handler)

In [3]:
pp = pprint.PrettyPrinter(width=119)

In [4]:
class LanguageModel(ABC):
    def __init__(self, model_name: str, temperature=0.7, max_tokens=512, seed=0):
        self.model_name = model_name
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.seed = seed
        logger.debug(f"model_name: {model_name}, temperature: {temperature}, max_tokens: {max_tokens}, seed: {seed}")

    @abstractmethod
    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        pass

In [5]:
# You must set OPENAI_API_KEY in your environment variables.
class OpenAIAPI(LanguageModel):
    def __init__(self, model_name: str, temperature=0.7, max_tokens=512, seed=0, base_url=None, api_key=None):
        super().__init__(model_name, temperature=temperature, max_tokens=max_tokens, seed=seed)
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        logger.debug(f"base_url: {self.client.base_url}, api_key: {self.client.api_key[:3]}...")

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        return [
            self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                seed=self.seed,
            )
            .choices[0]
            .message.content
            or ""
            for messages in messages_batch
        ]

In [6]:
# class VLLMModel(LanguageModel):
#     def __init__(self, model_name: str, temperature=0.7, max_tokens=512, seed=0, stop=[], repetition_penalty=1.0):
#         super().__init__(model_name, temperature=temperature, max_tokens=max_tokens, seed=seed)
#         self.vllm = LLM(model_name)
#         self.sampling_params = SamplingParams(temperature=temperature, max_tokens=max_tokens, seed=seed, stop=stop, repetition_penalty=repetition_penalty)
#         logger.debug(f"sampling_params: {self.sampling_params}")

#     def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
#         tokenizer = self.vllm.get_tokenizer()
#         prompts = [tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True) for messages in messages_batch]
#         # logger.debug(f"prompts[0]: {prompts[0]}")
#         outputs = self.vllm.generate(prompts, sampling_params=self.sampling_params, use_tqdm=False)
#         # logger.debug(f"outputs[0].outputs[0].text: {outputs[0].outputs[0].text}")
#         return [o.outputs[0].text for o in outputs]

In [7]:
llm = OpenAIAPI("gpt-4o-mini", temperature=0.7, max_tokens=512, seed=0)
llm

model_name: gpt-4o-mini, temperature: 0.7, max_tokens: 512, seed: 0
base_url: https://api.openai.com/v1/, api_key: sk-...


In [8]:
# llm = VLLMModel("cyberagent/calm3-22b-chat", temperature=0.7, max_tokens=512, seed=0, stop=["<|endoftext|>", "<|im_end|>", "<|im_start|>"], repetition_penalty=1.1)
# llm

In [9]:
llm([[{"role": "user", "content": "Hello, how are you?"}]])

["Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"]

In [10]:
def get_q1_prompt(task: str, topic: str, item: str, target: str) -> list[dict[str, str]]:
    Q1_PROMPT_TEMPLATE = """Create a {task} problem related to the following {topic}:

{item}

Note:

1. The {task} problem should be simple and involve basic {task} skills and knowledge. Any average {target} can solve it correctly.
2. You should make full use of the {topic} description to create the {task} problem to ensure that the {task} problem is unique and specific to the {topic}.
3. Your response should always start with "問題:". Your response should not include a solution to the created {task} problem.
4. 簡潔に日本語で回答してください。
"""
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": Q1_PROMPT_TEMPLATE.format(task=task, topic=topic, item=item, target=target)},
    ]

In [11]:
q1_prompt = get_q1_prompt("math", "persona", "SF作家", "grade school student")
pp.pprint(q1_prompt)

[{'content': 'Create a math problem related to the following persona:\n'
             '\n'
             'SF作家\n'
             '\n'
             'Note:\n'
             '\n'
             '1. The math problem should be simple and involve basic math skills and knowledge. Any average grade '
             'school student can solve it correctly.\n'
             '2. You should make full use of the persona description to create the math problem to ensure that the '
             'math problem is unique and specific to the persona.\n'
             '3. Your response should always start with "問題:". Your response should not include a solution to the '
             'created math problem.\n'
             '4. 簡潔に日本語で回答してください。\n',
  'role': 'user'}]


In [12]:
q1s = llm([get_q1_prompt("math", "persona", "SF作家", "grade school student")])
q1s

['問題: SF作家の佐藤さんは新しい短編小説を書いています。彼は毎日、物語の内容を考えるのに3時間を費やし、さらに2時間を執筆に使っています。もし彼が1週間（7日間）毎日同じ時間を使った場合、彼は合計で何時間を物語の考案と執筆に費やすことになりますか？']

In [13]:
q1s = llm([get_q1_prompt("advanced math", "persona", "SF作家", "graduate student")])
q1s

['問題: \n\nSF作家であるあなたは、未来の宇宙旅行の物語を執筆しています。あなたの物語の中で、宇宙船が光速の80%で移動すると仮定します。ある惑星から別の惑星までの距離は、地球からの距離で200光年です。相対性理論に基づいて、宇宙船が目的地に到達するまでの時間を、地球側の観測者と宇宙船の乗組員のそれぞれにとってどれくらいか計算してください。なお、相対性理論の時間の遅れの公式を用いることを忘れずに。']

In [14]:
def filter_q1(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^問題[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^Problem[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"\n答え[：:].*", "", content, flags=re.DOTALL)
    content = re.sub(r"\n[解回]答[：:].*", "", content, flags=re.DOTALL)
    content = re.sub(r"\n[Aa]nswer[：:].*", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

In [15]:
filter_q1(q1s[0])

'SF作家であるあなたは、未来の宇宙旅行の物語を執筆しています。あなたの物語の中で、宇宙船が光速の80%で移動すると仮定します。ある惑星から別の惑星までの距離は、地球からの距離で200光年です。相対性理論に基づいて、宇宙船が目的地に到達するまでの時間を、地球側の観測者と宇宙船の乗組員のそれぞれにとってどれくらいか計算してください。なお、相対性理論の時間の遅れの公式を用いることを忘れずに。'

In [16]:
filter_q1("問題: ここは問題です。\n\n解答: ここは答えです。")

'ここは問題です。'

In [17]:
def generate_q1(
    llm: LanguageModel, tasks: list[str], topics: list[str], items: list[str], targets: list[str]
) -> list[str]:
    return [
        filter_q1(q1)
        for q1 in llm(
            [
                get_q1_prompt(task, topic, item, target)
                for task, topic, item, target in zip(tasks, topics, items, targets)
            ]
        )
    ]

In [18]:
q1s = generate_q1(
    llm,
    ["math", "advanced math"],
    ["persona", "persona"],
    ["SF作家", "SF作家"],
    ["grade school student", "college student"],
)
q1s

['SF作家の佐藤さんは新しい短編小説を書いています。彼は毎日、物語の内容を考えるのに3時間を費やし、さらに2時間を執筆に使っています。もし彼が1週間（7日間）毎日同じ時間を使った場合、彼は合計で何時間を物語の考案と執筆に費やすことになりますか？',
 'SF作家であるあなたは、未来の宇宙旅行の物語を執筆しています。あなたの物語の中で、宇宙船が光速の80%で移動すると仮定します。ある惑星から別の惑星までの距離は、地球からの距離で200光年です。宇宙船の速度に基づいて、宇宙船がこの距離を移動するのにかかる時間（地球の観測者から見た時間）を計算してください。ただし、相対性理論を考慮し、時間の遅れを考慮する必要があります。具体的には、ローレンツ因子を使用して、地球での経過時間を求めてください。']

In [19]:
def get_a1_prompt(q1: str) -> list[dict]:
    A1_PROMPT_TEMPLATE = "{q1}\n\n簡潔に日本語で回答してください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": A1_PROMPT_TEMPLATE.format(q1=q1)},
    ]

In [20]:
a1_prompt = get_a1_prompt(q1s[0])
a1_prompt

[{'role': 'user',
  'content': 'SF作家の佐藤さんは新しい短編小説を書いています。彼は毎日、物語の内容を考えるのに3時間を費やし、さらに2時間を執筆に使っています。もし彼が1週間（7日間）毎日同じ時間を使った場合、彼は合計で何時間を物語の考案と執筆に費やすことになりますか？\n\n簡潔に日本語で回答してください。'}]

In [21]:
llm([get_a1_prompt(q1s[0])])

['佐藤さんは毎日5時間（3時間の考案 + 2時間の執筆）を費やしています。1週間で7日間あるので、合計は5時間 × 7日 = 35時間です。したがって、彼は合計で35時間を物語の考案と執筆に費やすことになります。']

In [22]:
llm([get_a1_prompt(q1s[1])])

['宇宙船が光速の80%（0.8c）で移動する場合、ローレンツ因子（γ）は次のように計算されます。\n\n\\[\n\\gamma = \\frac{1}{\\sqrt{1 - v^2/c^2}} = \\frac{1}{\\sqrt{1 - (0.8)^2}} = \\frac{1}{\\sqrt{1 - 0.64}} = \\frac{1}{\\sqrt{0.36}} = \\frac{1}{0.6} \\approx 1.6667\n\\]\n\n次に、200光年の距離を0.8cで移動するのにかかる地球の時間（t）を求めます。\n\n\\[\nt = \\frac{距離}{速度} = \\frac{200 \\text{光年}}{0.8c} = \\frac{200}{0.8} \\text{年} = 250 \\text{年}\n\\]\n\n相対性理論による時間の遅れを考慮するため、地球の観測者から見た経過時間は次のように修正されます。\n\n\\[\nt_{地球} = \\gamma \\times t_{宇宙船} = t\n\\]\n\nここで、宇宙船の時間（t_{宇宙船}）は以下のように求めます。\n\n\\[\nt_{宇宙船} = \\frac{t_{地球}}{\\gamma} = \\frac{250 \\text{年}}{1.6667} \\approx 150 \\text{年}\n\\]\n\nしたがって、地球の観測者から見た宇宙船が200光年を移動するのにかかる時間は約250年です。']

In [23]:
def filter_a1(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^答え[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[解回]答[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[Aa]nswer[：:]", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

In [24]:
filter_a1("解答: ここは答えです。")

'ここは答えです。'

In [25]:
def generate_a1(llm: LanguageModel, q1s: list[str]) -> list[str]:
    return [filter_a1(a1) for a1 in llm([get_a1_prompt(q1) for q1 in q1s])]

In [26]:
a1s = generate_a1(llm, q1s)
a1s

['佐藤さんは毎日5時間（3時間の考案 + 2時間の執筆）を費やしています。1週間で7日間あるので、合計は5時間 × 7日 = 35時間です。したがって、彼は合計で35時間を物語の考案と執筆に費やすことになります。',
 '宇宙船の速度が光速の80%（0.8c）であると仮定します。ローレンツ因子（γ）は以下の式で計算できます。\n\n\\[\n\\gamma = \\frac{1}{\\sqrt{1 - \\left(\\frac{v}{c}\\right)^2}} \n\\]\n\nここで、\\( v = 0.8c \\) なので、\n\n\\[\n\\gamma = \\frac{1}{\\sqrt{1 - (0.8)^2}} = \\frac{1}{\\sqrt{1 - 0.64}} = \\frac{1}{\\sqrt{0.36}} = \\frac{1}{0.6} \\approx 1.6667\n\\]\n\n次に、宇宙船が移動する距離は200光年です。地球の観測者から見た時間（\\( t \\)）は以下の式で求められます。\n\n\\[\nt = \\frac{d}{v} = \\frac{200 \\text{光年}}{0.8c}\n\\]\n\n光速の定義から、光年は時間の単位（1光年 = 1年で光が進む距離）として使われるため、\n\n\\[\nt = \\frac{200}{0.8} = 250 \\text{年}\n\\]\n\nしたがって、地球の観測者から見た宇宙船が200光年の距離を移動するのにかかる時間は250年です。']

In [27]:
def get_q2_prompt(q1: str, a1: str) -> list[dict[str, str]]:
    Q2_PROMPT_TEMPLATE = "前述の問題をより理解するために、簡潔な追加の質問を一つ作ってください。問題の一部を変更したり、条件を追加しても良いです。追加の質問だけを書き、決して答えを含めないでください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": q1},
        {"role": "assistant", "content": a1},
        {"role": "user", "content": Q2_PROMPT_TEMPLATE},
    ]

In [28]:
q2_prompt = get_q2_prompt(q1s[0], a1s[0])
q2_prompt

[{'role': 'user',
  'content': 'SF作家の佐藤さんは新しい短編小説を書いています。彼は毎日、物語の内容を考えるのに3時間を費やし、さらに2時間を執筆に使っています。もし彼が1週間（7日間）毎日同じ時間を使った場合、彼は合計で何時間を物語の考案と執筆に費やすことになりますか？'},
 {'role': 'assistant',
  'content': '佐藤さんは毎日5時間（3時間の考案 + 2時間の執筆）を費やしています。1週間で7日間あるので、合計は5時間 × 7日 = 35時間です。したがって、彼は合計で35時間を物語の考案と執筆に費やすことになります。'},
 {'role': 'user',
  'content': '前述の問題をより理解するために、簡潔な追加の質問を一つ作ってください。問題の一部を変更したり、条件を追加しても良いです。追加の質問だけを書き、決して答えを含めないでください。'}]

In [29]:
llm([get_q2_prompt(q1s[0], a1s[0])])

['もし佐藤さんが毎日、考案に使う時間を1時間増やし、執筆に使う時間を1時間減らした場合、1週間で合計何時間を物語の考案と執筆に費やすことになりますか？']

In [30]:
def filter_q2(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^追加の質問[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^質問[：:]", "", content, flags=re.DOTALL)
    content = content.strip()
    return content

In [31]:
def generate_q2(llm: LanguageModel, q1s: list[str], a1s: list[str]) -> list[str]:
    return [filter_q2(q2) for q2 in llm([get_q2_prompt(q1, a1) for q1, a1 in zip(q1s, a1s)])]

In [32]:
q2s = generate_q2(llm, q1s, a1s)
q2s

['もし佐藤さんが毎日、考案に使う時間を4時間、執筆に使う時間を1時間に変更した場合、1週間で合計何時間を物語の考案と執筆に費やすことになりますか？',
 '宇宙船が光速の90%（0.9c）で移動する場合、同じ200光年の距離を移動するのにかかる時間（地球の観測者から見た時間）を計算してください。ただし、ローレンツ因子を使用して、時間の遅れも考慮してください。']

In [33]:
list(zip(q1s, a1s, q2s))[0]

('SF作家の佐藤さんは新しい短編小説を書いています。彼は毎日、物語の内容を考えるのに3時間を費やし、さらに2時間を執筆に使っています。もし彼が1週間（7日間）毎日同じ時間を使った場合、彼は合計で何時間を物語の考案と執筆に費やすことになりますか？',
 '佐藤さんは毎日5時間（3時間の考案 + 2時間の執筆）を費やしています。1週間で7日間あるので、合計は5時間 × 7日 = 35時間です。したがって、彼は合計で35時間を物語の考案と執筆に費やすことになります。',
 'もし佐藤さんが毎日、考案に使う時間を4時間、執筆に使う時間を1時間に変更した場合、1週間で合計何時間を物語の考案と執筆に費やすことになりますか？')

In [34]:
def get_a2_prompt(q1: str, a1: str, q2: str) -> list[dict[str, str]]:
    A2_PROMPT_TEMPLATE = "{q2}\n\n簡潔に日本語で回答してください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": q1},
        {"role": "assistant", "content": a1},
        {"role": "user", "content": A2_PROMPT_TEMPLATE.format(q2=q2)},
    ]

In [35]:
a2_prompt = get_a2_prompt(q1s[0], a1s[0], q2s[0])
a2_prompt

[{'role': 'user',
  'content': 'SF作家の佐藤さんは新しい短編小説を書いています。彼は毎日、物語の内容を考えるのに3時間を費やし、さらに2時間を執筆に使っています。もし彼が1週間（7日間）毎日同じ時間を使った場合、彼は合計で何時間を物語の考案と執筆に費やすことになりますか？'},
 {'role': 'assistant',
  'content': '佐藤さんは毎日5時間（3時間の考案 + 2時間の執筆）を費やしています。1週間で7日間あるので、合計は5時間 × 7日 = 35時間です。したがって、彼は合計で35時間を物語の考案と執筆に費やすことになります。'},
 {'role': 'user',
  'content': 'もし佐藤さんが毎日、考案に使う時間を4時間、執筆に使う時間を1時間に変更した場合、1週間で合計何時間を物語の考案と執筆に費やすことになりますか？\n\n簡潔に日本語で回答してください。'}]

In [36]:
a2s = llm([get_a2_prompt(q1, a1, q2) for q1, a1, q2 in zip(q1s, a1s, q2s)])
a2s

['毎日5時間（4時間の考案 + 1時間の執筆）を費やします。1週間で7日間なので、合計は5時間 × 7日 = 35時間です。佐藤さんは合計で35時間を費やします。',
 '宇宙船が光速の90%（0.9c）で移動する場合、ローレンツ因子（γ）は以下のように計算されます。\n\n\\[\n\\gamma = \\frac{1}{\\sqrt{1 - (0.9)^2}} = \\frac{1}{\\sqrt{1 - 0.81}} = \\frac{1}{\\sqrt{0.19}} \\approx 2.294\n\\]\n\n次に、地球の観測者から見た時間（\\( t \\)）を計算します。\n\n\\[\nt = \\frac{d}{v} = \\frac{200 \\text{光年}}{0.9c} = \\frac{200}{0.9} \\approx 222.22 \\text{年}\n\\]\n\nしたがって、地球の観測者から見た宇宙船が200光年の距離を移動するのにかかる時間は約222.22年です。']

In [37]:
def filter_a2(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^答え[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[解回]答[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[Aa]nswer[：:]", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

In [38]:
def generate_a2(llm: LanguageModel, q1s: list[str], a1s: list[str], q2s: list[str]) -> list[str]:
    return [filter_a2(a2) for a2 in llm([get_a2_prompt(q1, a1, q2) for q1, a1, q2 in zip(q1s, a1s, q2s)])]

In [39]:
a2s = generate_a2(llm, q1s, a1s, q2s)
a2s

['毎日5時間（4時間の考案 + 1時間の執筆）を費やします。1週間で7日間なので、合計は5時間 × 7日 = 35時間です。佐藤さんは合計で35時間を費やします。',
 '宇宙船が光速の90%（0.9c）で移動する場合、ローレンツ因子（γ）は以下のように計算されます。\n\n\\[\n\\gamma = \\frac{1}{\\sqrt{1 - (0.9)^2}} = \\frac{1}{\\sqrt{1 - 0.81}} = \\frac{1}{\\sqrt{0.19}} \\approx 2.294\n\\]\n\n次に、地球の観測者から見た時間（\\( t \\)）を計算します。\n\n\\[\nt = \\frac{d}{v} = \\frac{200 \\text{光年}}{0.9c} = \\frac{200}{0.9} \\approx 222.22 \\text{年}\n\\]\n\nしたがって、地球の観測者から見た宇宙船が200光年の距離を移動するのにかかる時間は約222.22年です。']

In [40]:
list(zip(q1s, a1s, q2s, a2s))[0]

('SF作家の佐藤さんは新しい短編小説を書いています。彼は毎日、物語の内容を考えるのに3時間を費やし、さらに2時間を執筆に使っています。もし彼が1週間（7日間）毎日同じ時間を使った場合、彼は合計で何時間を物語の考案と執筆に費やすことになりますか？',
 '佐藤さんは毎日5時間（3時間の考案 + 2時間の執筆）を費やしています。1週間で7日間あるので、合計は5時間 × 7日 = 35時間です。したがって、彼は合計で35時間を物語の考案と執筆に費やすことになります。',
 'もし佐藤さんが毎日、考案に使う時間を4時間、執筆に使う時間を1時間に変更した場合、1週間で合計何時間を物語の考案と執筆に費やすことになりますか？',
 '毎日5時間（4時間の考案 + 1時間の執筆）を費やします。1週間で7日間なので、合計は5時間 × 7日 = 35時間です。佐藤さんは合計で35時間を費やします。')

In [41]:
list(zip(q1s, a1s, q2s, a2s))[1]

('SF作家であるあなたは、未来の宇宙旅行の物語を執筆しています。あなたの物語の中で、宇宙船が光速の80%で移動すると仮定します。ある惑星から別の惑星までの距離は、地球からの距離で200光年です。宇宙船の速度に基づいて、宇宙船がこの距離を移動するのにかかる時間（地球の観測者から見た時間）を計算してください。ただし、相対性理論を考慮し、時間の遅れを考慮する必要があります。具体的には、ローレンツ因子を使用して、地球での経過時間を求めてください。',
 '宇宙船の速度が光速の80%（0.8c）であると仮定します。ローレンツ因子（γ）は以下の式で計算できます。\n\n\\[\n\\gamma = \\frac{1}{\\sqrt{1 - \\left(\\frac{v}{c}\\right)^2}} \n\\]\n\nここで、\\( v = 0.8c \\) なので、\n\n\\[\n\\gamma = \\frac{1}{\\sqrt{1 - (0.8)^2}} = \\frac{1}{\\sqrt{1 - 0.64}} = \\frac{1}{\\sqrt{0.36}} = \\frac{1}{0.6} \\approx 1.6667\n\\]\n\n次に、宇宙船が移動する距離は200光年です。地球の観測者から見た時間（\\( t \\)）は以下の式で求められます。\n\n\\[\nt = \\frac{d}{v} = \\frac{200 \\text{光年}}{0.8c}\n\\]\n\n光速の定義から、光年は時間の単位（1光年 = 1年で光が進む距離）として使われるため、\n\n\\[\nt = \\frac{200}{0.8} = 250 \\text{年}\n\\]\n\nしたがって、地球の観測者から見た宇宙船が200光年の距離を移動するのにかかる時間は250年です。',
 '宇宙船が光速の90%（0.9c）で移動する場合、同じ200光年の距離を移動するのにかかる時間（地球の観測者から見た時間）を計算してください。ただし、ローレンツ因子を使用して、時間の遅れも考慮してください。',
 '宇宙船が光速の90%（0.9c）で移動する場合、ローレンツ因子（γ）は以下のように計算されます。\n\n\\[\n\\gamma = \\frac{1}{\\sqrt{1 - 

In [42]:
def synthesis_multi_turn_qa(
    llm: LanguageModel, tasks: list[str], topics: list[str], items: list[str], targets: list[str]
) -> list[dict[str, str | list[dict[str, str]]]]:
    q1s = generate_q1(llm, tasks, topics, items, targets)
    a1s = generate_a1(llm, q1s)
    q2s = generate_q2(llm, q1s, a1s)
    a2s = generate_a2(llm, q1s, a1s, q2s)
    return [
        {
            "messages": [
                {"role": "user", "content": q1},
                {"role": "assistant", "content": a1},
                {"role": "user", "content": q2},
                {"role": "assistant", "content": a2},
            ],
            "task": task,
            "topic": topic,
            "item": item,
            "target": target,
        }
        for task, topic, item, target, q1, a1, q2, a2 in zip(tasks, topics, items, targets, q1s, a1s, q2s, a2s)
    ]

In [43]:
qas = synthesis_multi_turn_qa(
    llm,
    ["math", "advanced math"],
    ["persona", "persona"],
    ["SF作家", "SF作家"],
    ["grade school student", "college student"],
)
qas

[{'messages': [{'role': 'user',
    'content': 'SF作家の佐藤さんは新しい短編小説を書いています。彼は毎日、物語の内容を考えるのに3時間を費やし、さらに2時間を執筆に使っています。もし彼が1週間（7日間）毎日同じ時間を使った場合、彼は合計で何時間を物語の考案と執筆に費やすことになりますか？'},
   {'role': 'assistant',
    'content': '佐藤さんは毎日5時間（3時間の考案 + 2時間の執筆）を費やします。1週間で7日間あるので、合計は5時間 × 7日 = 35時間です。'},
   {'role': 'user',
    'content': 'もし佐藤さんが週末にそれぞれ4時間を物語の考案に使い、平日は変わらず5時間を使った場合、1週間で合計何時間を物語の考案と執筆に費やすことになりますか？'},
   {'role': 'assistant',
    'content': '平日（5日間）は毎日5時間で、合計25時間。週末（2日間）は毎日4時間で、合計8時間。したがって、1週間で合計33時間を費やします。'}],
  'task': 'math',
  'topic': 'persona',
  'item': 'SF作家',
  'target': 'grade school student'},
 {'messages': [{'role': 'user',
    'content': 'SF作家の田中さんは、3つの異なる宇宙を舞台とした小説を執筆しています。それぞれの宇宙には異なる数の惑星があります。宇宙Aには8つの惑星、宇宙Bには12の惑星、宇宙Cには15の惑星があります。田中さんは、それぞれの宇宙から1つの惑星を選んで物語に登場させることにしました。田中さんが選ぶことができる惑星の組み合わせの総数を求めなさい。'},
   {'role': 'assistant',
    'content': '田中さんが選べる惑星の組み合わせの総数は、宇宙A、宇宙B、宇宙Cそれぞれの惑星の数を掛け合わせて求めます。\n\n宇宙A: 8つの惑星  \n宇宙B: 12の惑星  \n宇宙C: 15の惑星  \n\n組み合わせの総数は、  \n8 × 12 × 15 = 1440

In [44]:
# Here is a list of occupations. Extract items as a Python list.
# (Paste the content of https://ja.wikipedia.org/wiki/%E8%81%B7%E6%A5%AD%E4%B8%80%E8%A6%A7 )

personas = [
    "アイドル",
    "アーキビスト",
    "アクチュアリー",
    "アシスタントディレクター",
    "アスレティックトレーナー",
    "アーティスト",
    "アートディレクター",
    "アナウンサー",
    "アニメーター",
    "海人",
    "アメリカンフットボール選手",
    "アレンジャー",
    "あん摩マッサージ指圧師",
    "医師",
    "石工",
    "イタコ",
    "板前",
    "鋳物工",
    "イラストレーター",
    "医療監視員",
    "医療事務員",
    "医療従事者",
    "医療保険事務",
    "刺青師",
    "インストラクター",
    "インダストリアルデザイナー",
    "インタープリター (自然)",
    "インテリアコーディネーター",
    "インテリアデザイナー",
    "ウェディングプランナー",
    "ウェブデザイナー",
    "鵜飼い",
    "浮世絵師",
    "宇宙飛行士",
    "占い師",
    "運転士",
    "運転手",
    "運転代行",
    "映画監督",
    "映画スタッフ",
    "映画俳優",
    "映画プロデューサー",
    "営業員",
    "衛視",
    "衛生検査技師",
    "映像作家",
    "栄養教諭",
    "栄養士",
    "駅員",
    "駅長",
    "エクステリアデザイナー",
    "エグゼクティブ・プロデューサー",
    "絵師",
    "エステティシャン",
    "エディトリアルデザイナー",
    "絵本作家",
    "演歌歌手",
    "園芸家",
    "エンジニア",
    "演出家",
    "演奏家",
    "オートレース選手",
    "オプトメトリスト",
    "お笑い芸人",
    "お笑いタレント",
    "音楽家",
    "音楽評論家",
    "音楽プロデューサー",
    "音楽療法士",
    "音響監督",
    "音響技術者",
    "海技従事者",
    "会計士",
    "外交官",
    "外航客船パーサー",
    "介護ヘルパー",
    "海事代理士",
    "会社員",
    "海上自衛官",
    "海上保安官",
    "会長",
    "介助犬訓練士",
    "カイロプラクター",
    "カウンセラー",
    "画家",
    "学芸員",
    "科学者",
    "学者",
    "学生",
    "学長",
    "格闘家",
    "菓子製造技能士",
    "歌手",
    "歌人",
    "カスタマエンジニア",
    "楽器製作者",
    "学校事務職員",
    "学校職員",
    "学校用務員",
    "活動弁士",
    "家庭教師",
    "カーデザイナー",
    "歌舞伎役者",
    "カメラマン",
    "カラーコーディネーター",
    "カラーセラピスト",
    "為替ディーラー",
    "環境デザイナー",
    "環境計量士",
    "環境コンサルタント",
    "観光コンサルタント",
    "看護師",
    "看護助手",
    "鑑定人",
    "監督",
    "官房長官",
    "管理栄養士",
    "官僚",
    "議員",
    "機関士",
    "戯曲家",
    "起業家",
    "樵",
    "棋士 (囲碁)",
    "棋士 (将棋)",
    "記者",
    "騎手",
    "技術コンサルタント",
    "技術者",
    "気象予報士",
    "機長",
    "キックボクサー",
    "着付師",
    "客室乗務員",
    "脚本家",
    "キャビンアテンダント",
    "キャラクターデザイナー",
    "キャリア (国家公務員)",
    "キャリア・コンサルタント",
    "救急救命士",
    "救急隊員",
    "きゅう師",
    "給仕人",
    "厩務員",
    "キュレーター",
    "教育関係職員",
    "教員",
    "行政官",
    "行政書士",
    "競艇選手",
    "教頭",
    "教諭",
    "銀行員",
    "空間情報コンサルタント",
    "空間デザイナー",
    "グラウンドキーパー",
    "グラフィックデザイナー",
    "グランドスタッフ",
    "グランドホステス",
    "クリエイティブ・ディレクター",
    "クリーニング師",
    "クレーン運転士",
    "軍事評論家",
    "軍人",
    "ケアワーカー（介護士）",
    "経営コンサルタント",
    "経営者",
    "芸妓",
    "経済評論家",
    "警察官",
    "芸術家",
    "芸人",
    "芸能人",
    "芸能リポーター",
    "警備員",
    "刑務官",
    "警務官",
    "計量士",
    "競輪選手",
    "劇作家",
    "ケースワーカー",
    "ゲームクリエイター",
    "ゲームシナリオライター",
    "ゲームデザイナー",
    "ゲームライター",
    "検疫官",
    "研究員",
    "言語聴覚士",
    "検察官",
    "検察事務官",
    "建設コンサルタント",
    "現像技師",
    "建築家",
    "建築コンサルタント",
    "建築士",
    "校閲者",
    "航海士",
    "公共政策コンサルタント",
    "工業デザイナー",
    "航空管制官",
    "航空機関士",
    "皇宮護衛官",
    "航空自衛官",
    "航空従事者",
    "航空整備士",
    "工芸家",
    "講師 (教育)",
    "工場長",
    "交渉人",
    "講談師",
    "校長",
    "交通指導員",
    "高等学校教員",
    "公認会計士",
    "公務員",
    "校務員",
    "港湾荷役作業員",
    "国際公務員",
    "国連職員",
    "国税専門官",
    "国務大臣",
    "ゴーストライター",
    "国会議員",
    "国会議員政策担当秘書",
    "国会職員",
    "国家公務員",
    "コック",
    "コ・デンタル",
    "コピーライター",
    "コミッショナー",
    "コメディアン",
    "コ・メディカル",
    "コラムニスト",
    "顧問",
    "コンサルタント",
    "コンシェルジュ",
    "コンセプター",
    "コンピュータ技術者",
    "再開発プランナー・再開発コンサルタント",
    "裁判官",
    "裁判所職員",
    "裁判所調査官",
    "サウンドクリエイター",
    "左官",
    "作業療法士",
    "作詞家",
    "撮影監督",
    "撮影技師",
    "作家",
    "サッカー選手",
    "作曲家",
    "茶道家",
    "サラリーマン",
    "参議院議員",
    "指圧師",
    "自衛官",
    "シェフ",
    "歯科医師",
    "司会者",
    "歯科衛生士",
    "歯科技工士",
    "歯科助手",
    "士官",
    "指揮者",
    "司書",
    "司書教諭",
    "詩人",
    "システムアドミニストレータ",
    "システムエンジニア",
    "自然保護官",
    "質屋",
    "市町村長",
    "実業家",
    "自動車整備士",
    "児童文学作家",
    "シナリオライター",
    "視能訓練士",
    "司法書士",
    "事務員",
    "社会福祉士",
    "社会保険労務士",
    "車掌",
    "写真家",
    "写真ディレクター",
    "社長",
    "ジャーナリスト",
    "写譜屋",
    "獣医師",
    "衆議院議員",
    "臭気判定士",
    "柔道整復師",
    "守衛",
    "ジュエリーデザイナー",
    "塾講師",
    "手話通訳士",
    "准看護師",
    "准教授",
    "小学校教員",
    "上下水道コンサルタント",
    "証券アナリスト",
    "将校",
    "小説家",
    "消防官",
    "照明技師",
    "照明技術者",
    "照明士",
    "照明デザイナー",
    "書家",
    "助教",
    "助教授",
    "職人",
    "ショコラティエ",
    "助手 (教育)",
    "初生雛鑑別師",
    "書道家",
    "助産師",
    "シンガーソングライター",
    "神職",
    "審判員",
    "新聞記者",
    "新聞配達員",
    "心理カウンセラー",
    "診療放射線技師",
    "心理療法士",
    "森林コンサルタント",
    "樹医",
    "随筆家",
    "推理作家",
    "スカウト (勧誘)",
    "スクールカウンセラー",
    "寿司職人",
    "スタイリスト",
    "スタジオ・ミュージシャン",
    "スタント・パーソン",
    "スタントマン",
    "スチュワーデス",
    "スチュワード",
    "ストリートミュージシャン",
    "スパイ",
    "スーパーバイザー",
    "スポーツ選手",
    "スポーツドクター",
    "摺師",
    "製菓衛生師",
    "声楽家",
    "税関職員",
    "政治家",
    "聖職者",
    "整体師",
    "青年海外協力隊員",
    "整備士",
    "声優",
    "税理士",
    "セックスワーカー",
    "ゼネラルマネージャー",
    "セラピスト",
    "船員",
    "選挙屋",
    "船長",
    "戦場カメラマン",
    "染織家",
    "潜水士",
    "造園家/造園コンサルタント",
    "葬儀屋",
    "造形作家",
    "相場師",
    "操縦士",
    "装丁家",
    "僧侶",
    "測量士・測量技師",
    "ソーシャルワーカー",
    "速記士",
    "ソムリエ",
    "ソムリエール",
    "村議会議員",
    "大学教員",
    "大学教授",
    "大学職員",
    "大工",
    "大臣",
    "大道芸人",
    "大道芸人",
    "大統領",
    "ダイバー",
    "殺陣師",
    "旅芸人",
    "タレント",
    "ダンサー",
    "探偵",
    "チェリスト",
    "知事",
    "地質コンサルタント",
    "チーフプロデューサー",
    "地方議会議員",
    "地方公務員",
    "中学校教員",
    "中小企業診断士",
    "調教師",
    "調香師",
    "彫刻家",
    "聴導犬訓練士",
    "著作家",
    "ツアーコンダクター",
    "通関士",
    "通信士",
    "通訳",
    "通訳案内士",
    "ディスクジョッキー",
    "ディスパッチャー",
    "ディーラー",
    "ディレクター",
    "テクニカルディレクター (スポーツ)",
    "テクニカルディレクター (テレビ)",
    "テクノクラート",
    "デザイナー",
    "デザインプロデューサー",
    "テニス選手",
    "テレビディレクター",
    "テレビプロデューサー",
    "電気工事士",
    "電車運転士",
    "添乗員",
    "電話交換手",
    "陶芸家",
    "投資家",
    "杜氏",
    "動物看護師",
    "動物管理官",
    "時計師",
    "登山家",
    "都市計画コンサルタント",
    "図書館司書",
    "鳶職",
    "トラックメイカー",
    "トリマー",
    "ドリラー",
    "トレジャーハンター",
    "トレーナー",
    "内閣官房長官",
    "内閣総理大臣",
    "仲居",
    "ナニー",
    "ナレーター",
    "入国警備官",
    "入国審査官",
    "ニュースキャスター",
    "庭師",
    "塗師",
    "ネイリスト",
    "ネイルアーティスト",
    "ネットワークエンジニア",
    "農家",
    "能楽師",
    "納棺師",
    "農業土木コンサルタント",
    "ノンフィクション作家",
    "配管工",
    "俳人",
    "バイヤー",
    "俳優",
    "パイロット",
    "バスガイド",
    "バスケットボール選手",
    "パタンナー",
    "発明家",
    "パティシエ",
    "バーテンダー",
    "噺家",
    "花火師",
    "花屋",
    "はり師",
    "バリスタ (コーヒー)",
    "バルーンアーティスト",
    "パン屋",
    "ピアノ調律師",
    "美術 (職業)",
    "美術家",
    "美術商",
    "秘書",
    "筆跡鑑定人",
    "ビデオジョッキー",
    "ビューロクラート",
    "美容師",
    "評論家",
    "ビル管理技術者",
    "ファイナンシャル・プランナー",
    "ファシリテーター",
    "ファシリティマネジャー",
    "ファッションデザイナー",
    "ファッションフォトグラファー",
    "ファッションモデル",
    "ファンタジー作家",
    "ファンドマネージャー",
    "ファンドレイザー",
    "風俗嬢",
    "フェロー",
    "副校長",
    "服飾デザイナー",
    "副操縦士",
    "腹話術師",
    "舞台演出家",
    "舞台監督",
    "舞台俳優",
    "舞台美術家",
    "舞踏家",
    "武道家",
    "不動産鑑定士",
    "不動産屋",
    "フードコーディネーター",
    "舞踊家",
    "フライトアテンダント",
    "フラワーデザイナー",
    "プラントハンター",
    "ブリーダー",
    "振付師",
    "フリーライター",
    "プログラマ",
    "プロゴルファー",
    "プロジェクトマネージャ",
    "プロデューサー",
    "プロブロガー",
    "プロボウラー",
    "プロボクサー",
    "プロ野球選手",
    "プロレスラー",
    "文芸評論家",
    "文筆家",
    "フライス盤工",
    "ヘアメイクアーティスト",
    "ペスト・コントロール・オペレーター",
    "ベビーシッター",
    "編曲家",
    "弁護士",
    "編集者",
    "弁理士",
    "保安官",
    "保育士",
    "冒険家",
    "放射線技師",
    "宝飾デザイナー",
    "放送作家",
    "法務教官",
    "訪問介護員",
    "牧師",
    "保険計理人",
    "保健師",
    "保護観察官",
    "補償コンサルタント",
    "ホステス",
    "ホスト",
    "ボディーガード",
    "ホームヘルパー",
    "ホラー作家",
    "彫師",
    "翻訳家",
    "舞妓",
    "マジシャン (奇術)",
    "マーシャラー",
    "マスタリング・エンジニア",
    "マタギ",
    "マッサージ師",
    "マニピュレーター",
    "マルチタレント",
    "漫画家",
    "漫画原作者",
    "漫才師",
    "漫談家",
    "ミキサー",
    "巫女",
    "水先案内人",
    "水先人",
    "宮大工",
    "ミュージシャン",
    "無線通信士",
    "メイクアップアーティスト",
    "メイド",
    "メジャーリーガー",
    "盲導犬訓練士",
    "モデラー (模型)",
    "モデル (職業)",
    "薬剤師",
    "役者",
    "野菜ソムリエ",
    "郵便配達",
    "YouTuber",
    "洋菓子職人",
    "養護教諭",
    "洋裁師",
    "養蚕家",
    "幼稚園教員",
    "養蜂家",
    "ライトノベル作家",
    "ライフセービング",
    "落語家",
    "酪農家",
    "ラグビー選手",
    "ラジオパーソナリティ",
    "ランドスケープアーキテクト",
    "ランドスケーププランナー",
    "ランドスケープデザイナー",
    "ランドスケープコンサルタント",
    "理学療法士",
    "力士",
    "陸上自衛官",
    "リポーター",
    "猟師",
    "漁師",
    "理容師",
    "料理研究家",
    "料理人",
    "旅行作家",
    "林業従事者",
    "臨床検査技師",
    "臨床工学技士",
    "臨床心理士",
    "ルポライター",
    "レコーディング・エンジニア",
    "レーサー",
    "レーシングドライバー",
    "レスキュー隊員",
    "レポーター",
    "レンジャー",
    "労働基準監督官",
    "録音技師",
    "和菓子職人",
    "和裁士",
    "和紙職人",
    "A&R",
    "CMディレクター",
    "DJ",
    "ITコーディネータ",
    "MR",
    "PAエンジニア",
    "SF作家",
    "SP",
]

len(personas)

635

In [45]:
def run_synthesis_multi_turn_qa(
    llm: LanguageModel,
    num_batches: int,
    batch_size: int,
    output_jsonl: str,
    task_list: list[str],
    topic_list: list[str],
    item_list: list[str],
    target_list: list[str],
):
    with open(output_jsonl, "a", encoding="utf-8") as f:
        for i in range(num_batches):
            tasks = [random.choice(task_list) for _ in range(batch_size)]
            topics = [random.choice(topic_list) for _ in range(batch_size)]
            items = [random.choice(item_list) for _ in range(batch_size)]
            targets = [random.choice(target_list) for _ in range(batch_size)]
            qas = synthesis_multi_turn_qa(llm, tasks, topics, items, targets)
            for qa in qas:
                f.write(json.dumps(qa, ensure_ascii=False) + "\n")
                pp.pprint(qa)

In [46]:
run_synthesis_multi_turn_qa(
    llm,
    1,
    1,
    "output.jsonl",
    ["math", "arithmetic", "basic math", "basic arithmetic"],
    ["persona"],
    personas,
    ["grade school student"],
)

{'item': '照明士',
 'messages': [{'content': '照明士の佐藤さんは、1つの部屋に取り付けるために3つのライトを用意しました。もし佐藤さんがもう1つの部屋にも同じ数のライトを取り付ける場合、合計で何個のライトが必要になりますか？',
               'role': 'user'},
              {'content': '合計で6個のライトが必要です。', 'role': 'assistant'},
              {'content': '佐藤さんが3つのライトを用意した部屋の数が2つから4つに増えた場合、合計で何個のライトが必要になりますか？', 'role': 'user'},
              {'content': '合計で12個のライトが必要です。', 'role': 'assistant'}],
 'target': 'grade school student',
 'task': 'basic arithmetic',
 'topic': 'persona'}


In [47]:
run_synthesis_multi_turn_qa(
    llm,
    1,
    1,
    "output.jsonl",
    ["logical reasoning", "reasoning", "reasoning quiz", "reasoning game"],
    ["persona"],
    personas,
    ["grade school student"],
)

{'item': 'A&R',
 'messages': [{'content': 'A&R（アーティスト＆レパートリー）は、新しい音楽アーティストを見つけて契約する仕事をしています。彼は毎月、3人のアーティストに会います。今月、彼は以下のアーティストと会いました。\n'
                          '\n'
                          '1. アーティストAは、ポップ音楽を演奏しており、すでに5曲のヒット曲があります。\n'
                          '2. アーティストBは、インディーロックを演奏し、最近のアルバムが大好評で、音楽フェスティバルで注目されています。\n'
                          '3. アーティストCは、ジャズを演奏しており、まだデビューしていませんが、すでに多くのライブパフォーマンスを行っています。\n'
                          '\n'
                          'A&Rは、次の条件を考慮してアーティストを選ぶ必要があります：\n'
                          '- ヒット曲が多いアーティストを優先する。\n'
                          '- 新しい音楽スタイルを持つアーティストにも興味がある。\n'
                          '- デビューしていないアーティストにもチャンスを与えたい。\n'
                          '\n'
                          'A&Rは、どのアーティストを選ぶべきでしょうか？各アーティストの特徴を踏まえて、彼が選ぶアーティストの理由を考えてください。',
               'role': 'user'},
              {'content': 'A&Rは以下のアーティストを選ぶべきです：\n'
                          '\n'
                          '1. **アーティストA** - すでに5曲のヒット曲があり、商業的成功を収めているため、優先的に選ぶ

In [48]:
run_synthesis_multi_turn_qa(
    llm, 1, 1, "output.jsonl", ["coding"], ["topic"], ["ソーシャルメディア分析"], ["graduate student"]
)

{'item': 'ソーシャルメディア分析',
 'messages': [{'content': 'あるソーシャルメディアプラットフォームから取得した投稿データがあります。各投稿は、投稿者のユーザー名、投稿内容、いいね数、コメント数から構成されています。あなたの仕事は、指定されたユーザーの投稿データをフィルタリングし、そのユーザーが投稿したすべての投稿のいいね数とコメント数の合計を計算するプログラムを作成することです。\n'
                          '\n'
                          '入力として、投稿データのリスト（辞書形式）と、フィルタリングするユーザー名を受け取ります。出力は、そのユーザーの投稿に対するいいね数とコメント数の合計を含む辞書を返してください。\n'
                          '\n'
                          '例として、次のような投稿データがあります。\n'
                          '\n'
                          '```python\n'
                          'posts = [\n'
                          '    {"username": "user1", "content": "Hello World!", "likes": 10, "comments": 2},\n'
                          '    {"username": "user2", "content": "Good morning!", "likes": 5, "comments": 1},\n'
                          '    {"username": "user1", "content": "How are you?", "likes": 8, "comments": 3},\n'
                          ']\n'
                          '```\n'
                          '\n'
       